In [3]:
# Data Manipulation
import numpy as np

# Data Preprocessing
from sklearn.preprocessing import StandardScaler

# Model Training
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Performance Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve

#Plotting
import matplotlib.pyplot as plt
import plotly.graph_objs as go
#Hyperparameter Tunning
from sklearn.model_selection import GridSearchCV

#Dimensionality Reduction
from sklearn.decomposition import PCA 

# Load Data

In [5]:
# filepath = '/gpfs/wolf2/arm/atm124/world-shared/arm-summer-school-2024/machine_learning/2019_2021'
filepath = '../data/ML'
X = np.load(f'{filepath}/features_nsametC1.b1_rh_mean_2019_2021.npy')
y = np.load(f'{filepath}/label_nsametC1.b1_rh_mean_2019_2021.npy')
time = np.load(f'{filepath}/time_nsametC1.b1_rh_mean_2019_2021.npy')
data_windows = np.load(f'{filepath}/data_windows_nsametC1.b1_rh_mean_2019_2021.npy')

# Data Splitting

**Reference for train_test_split**\
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
random_state = 1265
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size=0.2, 
                                                    random_state=random_state,
                                                    stratify=y
                                                   )

# Model Training

## Logistic Regression

**Logistic Regression**\
Logistic regression, despite its name, is a linear model for classification rather than regression. Logistic regression is also known in the literature as logit regression, maximum-entropy classification (MaxEnt) or the log-linear classifier. In this model, the probabilities describing the possible outcomes of a single trial are modeled using a logistic function. The implementation of logistic regression in scikit-learn can be accessed from class LogisticRegression. This implementation can fit binary, One-vs- Rest, or multinomial logistic regression with optional L2 or L1 regularization.

**Reading Materials**\
https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

### Baseline Logistic Regression

In [ ]:
# Train a Logistic Regression model using training data
lr = LogisticRegression(random_state=5645, max_iter=200).fit(X_train, y_train)
# Make predictions using the trained LR model
y_train_pred_lr = lr.predict(X_train)
# Calculate confusion matrix with training data
conf_mx_lr = confusion_matrix(y_train, y_train_pred_lr)
conf_mx_lr

In [ ]:
# Evaluate model performance on testing data
y_test_pred_lr = lr.predict(X_test)
# Calculate confusion matrix with testing data
conf_mx_lr_test = confusion_matrix(y_test, y_test_pred_lr)
conf_mx_lr_test

In [ ]:
y_train_pred_prob_lr = lr.predict_proba(X_train)[:,1]
y_test_pred_prob_lr = lr.predict_proba(X_test)[:,1]

In [ ]:
def plot_performance_curves(y_train, y_train_pred_prob, y_test, y_test_pred_prob, postfix='NA'):
    
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_pred_prob, pos_label=1)
    precision_train, recall_train, _ = precision_recall_curve(y_train, y_train_pred_prob, pos_label=1)

    fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred_prob, pos_label=1)
    precision_test, recall_test, _ = precision_recall_curve(y_test, y_test_pred_prob, pos_label=1)
    
    roc_auc_train = auc(fpr_train, tpr_train)
    roc_auc_test  = auc(fpr_test, tpr_test)

    pr_auc_train  = auc(recall_train, precision_train)
    pr_auc_test   = auc(recall_test, precision_test)
    
    plt.figure(figsize=(12,5))
    plt.subplot(1, 2, 1)
    plt.plot(fpr_train, tpr_train, color='darkorange', lw=2, label='ROC curve (area = %0.4f) - Train' % roc_auc_train)
    plt.plot(fpr_train, tpr_train, color='orangered', lw=2, label='ROC curve (area = %0.4f) - Test' % roc_auc_test)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic (ROC) Curve - {postfix}')
    plt.legend(loc="lower right")
                   
    plt.subplot(1, 2, 2)
    plt.plot(recall_train, precision_train, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f) - Train' % pr_auc_train)
    plt.plot(recall_test, precision_test, color='aqua', lw=2, label='Precision-Recall curve (AUC = %0.2f) - Test' % pr_auc_test)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {postfix}')
    plt.legend(loc="lower left")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_performance_curves(y_train, y_train_pred_prob_lr, y_test, y_test_pred_prob_lr, 'Logistic Regresion')

### Penalized Logistic Regression

**Ridge**\
Ridge regression addresses some of the problems of Ordinary Least Squares by imposing a penalty on the size of coefficients. The ridge coefficients minimize a penalized residual sum of squares. 

**Lasso**\
The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer non-zero coefficients, effectively reducing the number of features upon which the given solution is dependent. For this reason, Lasso and its variants are fundamental to the field of compressed sensing. Under certain conditions, it can recover the exact set of non-zero coefficients.

**Elastic Net**\
ElasticNet is a linear regression model trained with L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. We control the convex combination of L1 and L2 using the l1_ratio parameter.
Elastic-net is useful when there are multiple features which are correlated with one another. Lasso is likely to pick one
of these at random, while elastic-net is likely to pick both.

**Reading Materials**\
[1] Ridge: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

[2] Lasso: https://scikit-learn.org/stable/modules/linear_model.html#lasso

[3] Elastic Net: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#lasso-and-elastic-net

In [ ]:
lr_penalized = LogisticRegression(random_state=5645,
                        max_iter=200,
                        penalty='l2', 
                        C=0.1).fit(X_train, y_train)
y_train_pred_lr_penalized = lr_penalized.predict(X_train)
np.unique(y_train_pred_lr_penalized, return_counts=True)

In [ ]:
conf_mx_lr_penalized = confusion_matrix(y_train, y_train_pred_lr_penalized)
conf_mx_lr_penalized

In [ ]:
y_test_pred_lr_penalized = lr_penalized.predict(X_test)
conf_mx_lr_penalized_test = confusion_matrix(y_test, y_test_pred_lr_penalized)
conf_mx_lr_penalized_test

In [ ]:
y_train_pred_prob_lr_penalized = lr_penalized.predict_proba(X_train)[:,1]
y_test_pred_prob_lr_penalized = lr_penalized.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_lr_penalized, y_test, y_test_pred_prob_lr_penalized, 'Penalized Logistic Regresion')

## Decision Tree

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation. 

Reading materials:

[1] https://scikit-learn.org/stable/modules/tree.html

[2] https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier(random_state=5645, criterion='gini')
dtree.fit(X_train, y_train)
y_train_pred_dtree = dtree.predict(X_train)
np.unique(y_train_pred_dtree, return_counts=True)

In [ ]:
conf_mx_dtree = confusion_matrix(y_train, y_train_pred_dtree)
conf_mx_dtree

In [ ]:
# Evaluate model performance on testing data
y_test_pred_dtree = dtree.predict(X_test)
# Calculate confusion matrix with testing data
conf_mx_dtree_test = confusion_matrix(y_test, y_test_pred_dtree)
conf_mx_dtree_test

In [ ]:
y_train_pred_prob_dtree = dtree.predict_proba(X_train)[:,1]
y_test_pred_prob_dtree = dtree.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_dtree, y_test, y_test_pred_prob_dtree, 'Decision Tree')

## Regularized Decision Tree

**How to restrict the growth of a Decision Tree**?

**Reading Materials**\
https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier


In [ ]:
dtree_regularized = DecisionTreeClassifier(random_state=5645, 
                                           criterion='gini', 
                                           max_depth=20, 
                                           min_samples_split=10,
                                           max_features='sqrt',
                                           min_samples_leaf = 10
                                          )
dtree_regularized.fit(X_train, y_train)
y_train_pred_dtree_regularized = dtree_regularized.predict(X_train)
np.unique(y_train_pred_dtree_regularized, return_counts=True)

In [ ]:
conf_mx_dtree_regularized = confusion_matrix(y_train, y_train_pred_dtree_regularized)
conf_mx_dtree_regularized

In [ ]:
# Evaluate model performance on testing data
y_test_pred_dtree_regularized = dtree_regularized.predict(X_test)
# Calculate confusion matrix with testing data
conf_mx_dtree_regularized_test = confusion_matrix(y_test, y_test_pred_dtree_regularized)
conf_mx_dtree_regularized_test

In [ ]:
y_train_pred_prob_dtree_regularized = dtree_regularized.predict_proba(X_train)[:,1]
y_test_pred_prob_dtree_regularized = dtree_regularized.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_dtree_regularized, y_test, y_test_pred_prob_dtree_regularized, 'Regularized Decision Tree')

## Random Forest

In random forests, each tree in the ensemble is built from a sample drawn with replacement (i.e., a bootstrap sample) from the training set.

Furthermore, when splitting each node during the construction of a tree, the best split is found through an exhaustive search of the features values of either all input features or a random subset of size max_features. 

The purpose of these two sources of randomness is to decrease the variance of the forest estimator. Indeed, individual decision trees typically exhibit high variance and tend to overfit. The injected randomness in forests yield decision trees with somewhat decoupled prediction errors. By taking an average of those predictions, some errors can cancel out. Random forests achieve a reduced variance by combining diverse trees, sometimes at the cost of a slight increase in bias. In practice the variance reduction is often significant hence yielding an overall better model.

**Reading Materials**\
[1] https://scikit-learn.org/stable/modules/ensemble.html#random-forests
[2] https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier

In [ ]:
rnd= RandomForestClassifier(n_jobs = -1, random_state=1110)
rnd.fit(X_train, y_train)
y_train_pred_rnd = rnd.predict(X_train)
conf_mx_rnd = confusion_matrix(y_train, y_train_pred_rnd)
conf_mx_rnd

In [ ]:
y_test_pred_rnd = rnd.predict(X_test)
conf_mx_rnd_test = confusion_matrix(y_test, y_test_pred_rnd)
conf_mx_rnd_test

In [ ]:
y_train_pred_prob_rnd = rnd.predict_proba(X_train)[:,1]
y_test_pred_prob_rnd = rnd.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_rnd, y_test, y_test_pred_prob_rnd, 'Random Forest')

## Random Forest - Hyperparameter Tunning

**Reading Materials**\
[1] 
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

[2] https://scikit-learn.org/stable/modules/classes.html#hyper-parameter-optimizers

In [ ]:
param_grid_rnd = [{
    'max_leaf_nodes':[50, 100],
    'min_samples_leaf':[1, 20],
    'n_estimators':[100, 500],
    'max_features':["sqrt", None],
    'criterion': ["entropy"]
}]

rnd_grid = GridSearchCV(rnd, param_grid_rnd, cv=3)
rnd_grid.fit(X_train, y_train)

In [ ]:
rnd_grid.best_estimator_, rnd_grid.best_params_

In [ ]:
y_train_pred_rnd_grid = rnd_grid.predict(X_train)
conf_mx_rnd_grid = confusion_matrix(y_train, y_train_pred_rnd_grid)
conf_mx_rnd_grid

In [ ]:
y_test_pred_rnd_grid = rnd_grid.predict(X_test)
conf_mx_rnd_grid_test = confusion_matrix(y_test, y_test_pred_rnd_grid)
conf_mx_rnd_grid_test

In [ ]:
y_train_pred_prob_rnd_grid = rnd_grid.predict_proba(X_train)[:,1]
y_test_pred_prob_rnd_grid = rnd_grid.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_rnd_grid, y_test, y_test_pred_prob_rnd_grid, 'Random Forest - 3 Fold CV')

## Use the best parameter values to build a Random Forest

In [ ]:
rnd_select = RandomForestClassifier(n_jobs = -1, 
                                      random_state=1110,
                                      max_leaf_nodes=100,
                                      min_samples_leaf=1,
                                      max_features = 'sqrt',
                                      n_estimators=500,
                                      criterion='entropy',
                                     )
rnd_select.fit(X_train, y_train)

In [ ]:
y_train_pred_rnd_select = rnd_select.predict(X_train)
np.unique(y_train_pred_rnd_select, return_counts=True)

In [ ]:
y_test_pred_rnd_select = rnd_select.predict(X_test)
np.unique(y_test_pred_rnd_select, return_counts=True)

In [ ]:
conf_mx_rnd_select = confusion_matrix(y_train, y_train_pred_rnd_select)
conf_mx_rnd_select

In [ ]:
conf_mx_rnd_select_test = confusion_matrix(y_test, y_test_pred_rnd_select)
conf_mx_rnd_select_test

In [ ]:
y_train_pred_prob_rnd_select = rnd_select.predict_proba(X_train)[:,1]
y_test_pred_prob_rnd_select = rnd_select.predict_proba(X_test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_rnd_select, y_test, y_test_pred_prob_rnd_select, 'Random Forest - Best Parameters')

# Dimensionality Reduction - PCA

**Principal component analysis (PCA)**

Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.

**Reading Materials**\
[1]https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA

[2]https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html#sphx-glr-auto-examples-decomposition-plot-pca-iris-py

## PCA for Visualization

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA()
transformed_X = pca.fit_transform(X_scaled)

In [ ]:
pca.explained_variance_ratio_, sum(pca.explained_variance_ratio_[:5])

In [ ]:
# fig = go.Figure(data=[go.Scatter3d(
#     x=transformed_X[:, 0],
#     y=transformed_X[:, 1],
#     z=transformed_X[:, 2],
#     mode='markers',
#     marker=dict(
#         size=2,
#         color=y,  # Color by class label
#         colorscale='Viridis',
#         opacity=0.5
#     ),
#     text=['Class: {}'.format(label) for label in y],
# )])

# # Set layout
# fig.update_layout(scene=dict(
#     xaxis_title='Principal Component 1',
#     yaxis_title='Principal Component 2',
#     zaxis_title='Principal Component 3',
# ), title='3D Scatter Plot of First 3 Principal Components')

# fig.update_layout(width=800, height=600)
# fig.show()

## PCA Model Training - Random Forest

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
pca_train = PCA(n_components=3)
transformed_X_Train = pca_train.fit_transform(X_train_scaled)
transformed_X_Test = pca_train.transform(X_test_scaled)
rnd_pca = RandomForestClassifier(n_jobs = -1, 
                                      random_state=1110,
                                      max_leaf_nodes=100,
                                      min_samples_leaf=1,
                                      max_features = 'sqrt',
                                      n_estimators=500,
                                      criterion='entropy'
                                     )
rnd_pca.fit(transformed_X_Train, y_train)

In [ ]:
y_train_pred_rnd_pca = rnd_pca.predict(transformed_X_Train)
np.unique(y_train_pred_rnd_pca, return_counts=True)

In [ ]:
y_test_pred_rnd_pca = rnd_pca.predict(transformed_X_Test)
np.unique(y_test_pred_rnd_pca, return_counts=True)

In [ ]:
conf_mx_rnd_pca = confusion_matrix(y_train, y_train_pred_rnd_pca)
conf_mx_rnd_pca

In [ ]:
conf_mx_rnd_pca_test = confusion_matrix(y_test, y_test_pred_rnd_pca)
conf_mx_rnd_pca_test

In [ ]:
y_train_pred_prob_rnd_pca = rnd_pca.predict_proba(transformed_X_Train)[:,1]
y_test_pred_prob_rnd_pca = rnd_pca.predict_proba(transformed_X_Test)[:,1]
plot_performance_curves(y_train, y_train_pred_prob_rnd_pca, y_test, y_test_pred_prob_rnd_pca, 'Random Forest - PCA')